# 要約 
このJupyter Notebookは、KaggleのLMSYS - Chatbot Arenaコンペティションにおいて、ユーザーの応答の好みを予測するための機械学習モデルを構築することに取り組んでいます。特に、二つの言語モデルの応答のうちどちらが好まれるかを予測するための特徴量を生成し、最終的にモデルの予測を結合するアプローチを取っています。

### 主要な手法とライブラリ
1. **使用ライブラリ**:
   - `transformers`: 言語モデルを扱うためのライブラリ（例: Llamaモデルを利用）。
   - `torch`: PyTorchでの深層学習。
   - `lightgbm`: 効率的な勾配ブースティングフレームワーク。
   - `pandas`: データ操作用。

2. **モデルの設定**:
   - Llamaモデル（LlamaForSequenceClassification）を利用し、CUDAデバイスを使用して8ビットでモデルを読み込み、メモリ効率を向上させる設定を施しています。
   - LoRa（Low-Rank Adaptation）方式を使ってモデルの重みを適用しており、推論時にモデルの評価を行います。

3. **データ準備と前処理**:
   - テストデータとサンプル提出ファイルを読み込み、応答を処理する関数を定義して、テキストを適切に整形します。
   - トークナイザーを用いてテキストをトークン化し、注意マスクを生成します。

4. **推論プロセス**:
   - 推論を効率的に行うためにバッチ処理を行い、複数のスレッドを用いて異なるGPU上でモデルを並行して実行します。
   - 各モデルから得られた確率をデータフレームに保存し、最終的な出力を構築します。

5. **特徴量生成**:
   - LightGBMとCountVectorizerを使用して特徴量を抽出し、応答の差異やプロンプトの統計的特性を計算します。

6. **予測の融合**:
   - LlamaモデルとLightGBMモデルからの予測をブレンドし、最終的な予測結果を生成します。

### 結果の保存
生成した予測結果は、`submission.csv`というファイル名で保存され、Kaggleに提出できる形式に整形されています。

このNotebookは、現実のユーザープロフィールや対話の特性を考慮した深層学習モデルと機械学習モデルの組み合わせにより、ユーザーの好みを予測するための実用的なアプローチを示しています。

---


# 用語概説 
以下は、機械学習・深層学習ノートブックに特有で、初心者がつまずきそうな専門用語の解説です。

1. **BitsAndBytesConfig**: これは、モデルのメモリ使用量を削減するための設定を管理するクラスです。特に、8ビットモードでのモデルの読み込みや計算を可能にし、メモリ効率を向上させます。

2. **LoRa (Low-Rank Adaptation)**: LoRaは、モデルの重みを更新するために使用される手法で、特に小さな計算コストでモデルの適応を行える方法です。この方法では、モデルの特定の部分を選択的に適応させ、全体の構造を保持しつつ新しいタスクへの微調整を行います。

3. **PeftModel/PeftConfig**: これらは、PEFT（Parameter-Efficient Fine-Tuning）に関連するクラスで、元のモデルのパラメータを必要最小限だけ変更して、新しいタスクに適応させるための設定を定義します。これにより、全てのパラメータを訓練するよりも効率よくモデルを適応させることができます。

4. **autocast**: PyTorchにおける自動混合精度（Automatic Mixed Precision）のためのコンテキストマネージャーです。異なる精度のデータ型間で計算を自動的に調整し、トレーニングや推論の速度を向上させると同時に、メモリの使用量も削減します。

5. **attention mask**: 自然言語処理においてトークンが有効か無効かを示すバイナリのマスクで、各トークンの重要度や有効性を制御します。特に、パディングトークンが無視されるように指定します。

6. ** symlog変換**: 対数変換の一種で、値がゼロや負の数でも計算できるようにするための変換方法です。負の値を適切に扱うため、数値の符号を保ちつつ対数を取ることができます。

7. **CountVectorizer**: テキストデータを数値データに変換するためのツールで、文書中の単語の出現回数をカウントし、特徴量として利用します。これは主に、機械学習アルゴリズムにテキスト情報を入力するために使用されます。

8. **Softmax**: 多クラス分類のための出力層で、モデルの出力を確率に変換する関数です。各クラスに属する確率を表すため、出力が正規化された状態になります。

これらの用語の理解は、ノートブック内で使われている技術と手法を理解するために重要です。

---


In [ ]:
# 必要なパッケージをインストールします
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/

このノートブックの作業は、以下のノートブックに触発されています：
* https://www.kaggle.com/code/ivanvybornov/llama3-8b-lgbm-tfidf
* https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b

## もしこれが役に立ったら、いいねを押してくれると嬉しいです

## ライブラリのインポート



In [ ]:
from threading import Thread
import gc
import os
import io
import json
import random
import pickle
import zipfile
import datetime
import time

import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from torch.cuda.amp import autocast
from IPython.display import display
import torch.nn.functional as F
import tokenizers

In [ ]:
# CUDAのメモリ効率を向上させる設定を有効にします
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)

MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # モデル名の指定
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'  # 重みのパス
MAX_LENGTH = 2048  # 最大シーケンス長
BATCH_SIZE = 4  # バッチサイズ
DEVICE = torch.device("cuda")  # GPUデバイスの指定

## データの準備



In [ ]:
# テストデータとサンプル提出ファイルを読み込む
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')

In [ ]:
# リスト内の文字列を連結する関数
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)  # 文をスペースで結合して返す

# 各カラムにprocess関数を適用
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

# サンプル提出ファイルとテストデータの先頭5行を表示
display(sample_sub)
display(test.head(5))

# モデル用のテキストを準備
test['text'] = 'User prompt: ' + test['prompt'] +  '\n\nModel A :\n' + test['response_a'] +'\n\n--------\n\nModel B:\n'  + test['response_b']
print(test['text'][0])  # テキストの最初の行を出力

## トークナイズ



In [ ]:
# トークナイザーの読み込み
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# テキストをトークン化し、パディングや切り捨てを適用
tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

# 入力IDと注意マスクをデバイスに移動
INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# テンソルをCPUに移動し、リストに変換
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

# データフレーム作成
data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu  # 入力IDを追加
data['ATTENTION_MASKS'] = attention_masks_cpu  # 注意マスクを追加
data[:2]  # 最初の2行を表示

## モデルの読み込み
> 各GPUに1つのモデルを読み込みます。



In [ ]:
# BitsAndBytesの設定
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットでの読み込みを有効に
    bnb_8bit_compute_dtype=torch.float16,  # 8ビット計算のデータ型
    bnb_8bit_use_double_quant=False)  # 二重量子化を無効に

# GPU 0にベースモデルを読み込み
device0 = torch.device('cuda:0')

base_model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # クラスラベル数
    torch_dtype=torch.float16,  # 計算に使用するデータ型
    quantization_config=bnb_config,  # 量子化設定
    device_map='cuda:0')  # デバイスマップの設定
base_model_0.config.pad_token_id = tokenizer.pad_token_id  # パディング用トークンIDを設定

In [ ]:
# GPU 1にベースモデルを読み込み
device1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # クラスラベル数
    torch_dtype=torch.float16,  # 計算に使用するデータ型
    quantization_config=bnb_config,  # 量子化設定
    device_map='cuda:1')  # デバイスマップの設定
base_model_1.config.pad_token_id = tokenizer.pad_token_id  # パディング用トークンIDを設定

## 重みの読み込み



In [ ]:
# LoRaの設定
peft_config = LoraConfig(
    r=16,  # ランク
    lora_alpha=32,  # LoRaのアルファ値
    lora_dropout=0.10,  # ドロップアウト率
    bias='none',  # バイアスの設定
    inference_mode=True,  # 推論モードを有効に
    task_type=TaskType.SEQ_CLS,  # タスクの種類
    target_modules=['o_proj', 'v_proj'])  # 対象モジュールの指定

In [ ]:
# PEFTモデルの取得
model_0 = get_peft_model(base_model_0, peft_config).to(device0) 
# 重みを読み込む
model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_0.eval()  # 評価モードに設定

model_1 = get_peft_model(base_model_1, peft_config).to(device1)
model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_1.eval()  # 評価モードに設定

# 学習可能なパラメータを出力
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()

In [ ]:
gc.collect()  # ガーベジコレクションを実行

## 推論



In [ ]:
def inference(df, model, device, batch_size=BATCH_SIZE):
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)  # 入力IDをテンソルに変換
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)  # 注意マスクをテンソルに変換
    
    generated_class_a = []  # モデルAの生成したクラス確率を保存
    generated_class_b = []  # モデルBの生成したクラス確率を保存
    generated_class_c = []  # タイのクラス確率を保存

    model.eval()  # 評価モードに設定
    
    for start_idx in range(0, len(df), batch_size):  # バッチ処理でデータを処理
        end_idx = min(start_idx + batch_size, len(df))
        batch_input_ids = input_ids[start_idx:end_idx].to(device)  # バッチの入力IDをデバイスに転送
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)  # バッチの注意マスクをデバイスに転送
        
        with torch.no_grad():  # 勾配計算を無効に
            with autocast():  # 自動混合精度を有効に
                outputs = model(  # モデルに入力IDと注意マスクを渡して出力を得る
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_mask
                )
        
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()  # 出力をソフトマックスで確率に変換
        
        generated_class_a.extend(probabilities[:, 0])  # モデルAの確率を追加
        generated_class_b.extend(probabilities[:, 1])  # モデルBの確率を追加
        generated_class_c.extend(probabilities[:, 2])  # タイの確率を追加
    
    df['winner_model_a'] = generated_class_a  # モデルAの勝者確率を追加
    df['winner_model_b'] = generated_class_b  # モデルBの勝者確率を追加
    df['winner_tie'] = generated_class_c  # タイの勝者確率を追加

    torch.cuda.empty_cache()  # CUDAキャッシュをクリア

    return df  # 結果を返す

In [ ]:
st = time.time()  # 処理開始時刻を記録

N_SAMPLES = len(data)  # サンプル数を取得

# データを2つのサブセットに分割
half = round(N_SAMPLES / 2)
sub1 = data.iloc[0:half].copy()  # 最初の半分をサブ1に
sub2 = data.iloc[half:N_SAMPLES].copy()  # 残りをサブ2に

# スレッドで推論を実行する関数
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)  # 推論結果を保存

# スレッド用のリストを作成
results = {}

In [ ]:
# スレッドの開始
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # サブ1の推論スレッド
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # サブ2の推論スレッド

t0.start()  # スレッドを開始
t1.start()  # スレッドを開始

# 全てのスレッドが完了するのを待つ
t0.join()
t1.join()

# 結果を元のデータフレームに結合
data = pd.concat([results[0], results[1]], axis=0)

print(f"処理が完了しました。トータル時間: {time.time() - st}")  # 処理完了時刻を出力

TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']  # ターゲット列を指定

sample_sub[TARGETS] = data[TARGETS]  # サンプル提出ファイルに予測値を追加

In [ ]:
llama_preds = data[TARGETS].values  # 予測値を取得

## LGBM + tfidf



In [ ]:
TAG = 'lmsys-chatbot-arena'  # コンペティションのタグ
RUNPOD = os.path.exists('/workspace/')  # 実行環境がワークポッドか確認
KAGGLE = not RUNPOD  # 実行環境がKaggleか確認
if KAGGLE: 
    print('kaggle')  # Kaggle環境である旨を出力

In [ ]:
try:
    import pandas as pd  # pandasをインポート
except:
    # 必要なパッケージがインポートできない場合、インストールする
    !pip install -q kaggle
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm 
    !pip install -q protobuf 
    !pip install -q numba

In [ ]:
# データパスの指定
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)

# ワークポッド環境の場合、Kaggle APIキーの設定
if RUNPOD:
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json
        !chmod 600 /root/.kaggle/kaggle.json

    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/  # コンペティションのデータをダウンロード
        
    if not os.path.exists('/data/'):
        import zipfile
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')  # ダウンロードしたzipファイルを展開

In [ ]:
INPUT_PATH = '/kaggle/input/'  # Kaggleの入力パス
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'  # モデルパスを設定
print(MODEL_PATH)  # モデルパスを出力

CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'  # コードパスの設定
SAVE_PATH = MODEL_PATH if not KAGGLE else ''  # 保存パスの設定

In [ ]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # トークナイザーの並列処理を無効に

In [ ]:
# データを読み込む
train = pd.read_csv(open(DATA + 'train.csv', 'r'))  # トレーニングデータを読み込み
test = pd.read_csv(open(DATA + 'test.csv', 'r'))  # テストデータを読み込み
sample = pd.read_csv(DATA + 'sample_submission.csv')  # サンプル提出データを読み込み
print(len(train), len(test))  # トレーニングデータとテストデータの行数を出力

In [ ]:
params = {}  # パラメータを初期化
if False: 
    pass;
    params['subsample'] = 30
else:
    params['fold'] = -1  # フォールド設定

params['n_epochs'] = 1  # エポック数を設定
params['n_lgb'] = 1  # LightGBMモデル数を設定
params['model'] = 'microsoft/deberta-v3-small'  # 使用するモデルを設定

In [ ]:
# パラメータを設定
FULL = params.get('fold', 0) < 0  # フルデータ使用フラグを設定
N_FOLDS = int(params.get('n_folds', 3));  # フォールド数を設定
FOLD = int(params.get('fold', 0))  # 現在のフォールドを設定
SEED = int(params.get('seed', 3))  # シード値を設定
SS = int(params.get('subsample', 1))  # サブサンプリング設定

print(N_FOLDS, FOLD, SEED, SS)  # 設定したパラメータを出力

In [ ]:
from sklearn.model_selection import StratifiedKFold

# Stratified K-Foldを使用してフォールドを取得する関数
def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state = SEED, shuffle = True)\
                    .split(X = np.zeros(len(train)), y = train.iloc[:, -3:].idxmax(1)))  # 表示用のフォールドを取得

train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]  # フォールドのIDを取得
if SS > 1:
    train_ids, test_ids = train_ids[::SS], test_ids[::SS]  # サブサンプリングを適用

print(len(train_ids), len(test_ids));  assert set(train_ids) & set(test_ids) == set()  # データが重複していないことを確認

In [ ]:
# ランダムシードを現在のマイクロ秒で設定
torch.manual_seed(datetime.datetime.now().microsecond)
random.seed(datetime.datetime.now().microsecond)
np.random.seed(datetime.datetime.now().microsecond)

In [ ]:
TRAIN = False  # トレーニングモードのフラグ
INFER = True   # 推論モードのフラグ 
SAVE = False   # 保存モードのフラグ

In [ ]:
import lightgbm as lgb  # LightGBMをインポート
from sklearn.feature_extraction.text import CountVectorizer  # 文書のカウントベクトル化をインポート

In [ ]:
LGB = True  # LightGBMを使用するフラグ
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0  # トレーニングフラグの設定
INFER_LGB = not TRAIN and LGB  # 推論フラグの設定

In [ ]:
# 事前に保存したCountVectorizerを読み込む
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))  # カウントベクトライザーを読み込み
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))  # 追加のカウントベクトライザーを読み込み

In [ ]:
# シンメトリック対数変換を定義
def symlog(x):
    return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)

# 行列を密な形式に変換し、symlog変換を適用
def dense(x):
    x = np.asarray(x.astype(np.float32).todense())
    x = symlog(x)
    return x

# 特徴量を取得する関数
def get_features(df):
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])  # プロンプトの特徴量
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])  # モデルAの応答の特徴量
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])  # モデルBの応答の特徴量
    
    v = np.hstack([
          afeat - bfeat, np.abs(afeat - bfeat),  # A応答とB応答の差を特徴量に追加
        ])
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)  # 投票モデルの数で正規化
    except:
        pass

    extras = []  # 追加の特徴量リスト
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']  # 追加の特徴量に使用する文字列
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)  # 各列での文字のカウント
    
    extras.append(df[c].str.len())  # 各列の長さを追加
    extras.append(df[c].str.split().apply(lambda x: len(x)))  # 各列の単語数を追加
    
    extras = np.stack(extras, axis = 1)  # 追加特徴量をスタック
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])  # 特徴量を平方根と対数変換
    return np.hstack([v, extras])  # 特徴量を結合して返す

In [ ]:
# 学習済みLightGBMモデルを読み込む
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))

In [ ]:
# 推論モードでLightGBMを使用する設定
if INFER and params.get('n_lgb', 1) > 0:
    df = test  # テストデータを使用
    yps = []  # 予測結果を格納するリスト
    b = 1000  # バッチサイズ
    for i in range(0, len(df), b):
        arr = get_features(df.iloc[i: i + b])  # 特徴量を取得
        ypms = []  # 各モデルの予測を格納するリスト
        for model in lgb_models:
            ypms.append(model.predict_proba(arr))  # 各モデルの確率を予測
        yps.append(np.stack(ypms).mean(0))  # 各モデルの平均確率を計算
        print('.', end = '')  # 進捗を表示
        
        if len(yps) % 2 == 0:  # 2回ごとにガーベジコレクションを実行
            gc.collect()
    print()

    yp = np.concatenate(yps)  # 予測結果を結合

In [ ]:
lgb_preds = yp  # LightGBMの予測結果を格納

## 予測のブレンド

$\operatorname{preds} = 0.05 \cdot \operatorname{lgbm \, boosting \, preds} + 0.8 \cdot \operatorname{llama \, preds}$



In [ ]:
lgb_wt = 0.05  # LightGBMの重みを設定
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds  # 重みに基づき予測をブレンド

In [ ]:
# 予測結果をデータフレームに変換
out = pd.DataFrame(preds, index=df.id, columns=train.columns[-3:])
display(out.head())  # 最初の5行を表示

In [ ]:
out.to_csv('submission.csv')  # 提出用ファイルに予測結果を保存